In [11]:
import pandas as pd
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
df.dropna(inplace=True)

In [13]:
X = df.drop('label', axis=1)

In [14]:
## Get the Dependent features
y=df['label']

### Genism Representation

#### import necessary libraries


In [15]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from gensim.models import Word2Vec
import numpy as np


#### add configurations

In [16]:
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
# Text Preprocessing function
def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    return words

#### do data preprocessing

In [18]:
# Apply preprocessing
X['processed_text'] = X['text'].apply(preprocess_text)

In [19]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [22]:
word2vec_model = Word2Vec(sentences=X['processed_text'], vector_size=100, window=5, min_count=1, workers=4)


In [23]:

# Function to get the average Word2Vec embedding for a sentence
def get_average_word2vec(text, model, vector_size):
    vec = np.zeros(vector_size)
    count = 0
    for word in text:
        if word in model.wv.key_to_index:
            vec += model.wv[word]
            count += 1
    if count != 0:
        vec /= count
    return vec

# Generate embeddings for each text
vector_size = word2vec_model.vector_size
X['embedding'] = X['processed_text'].apply(lambda x: get_average_word2vec(x, word2vec_model, vector_size))

In [24]:
X.head()

,id,title,author,text,processed_text,embedding
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,"[house, dem, aide, even, see, comey, letter, j...","[-0.7067920069261443, 0.3983624455672261, -0.6..."
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"[ever, get, feeling, life, circles, roundabout...","[-0.5550385904586405, 0.9413301657640748, -0.6..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...","[truth, might, get, fired, october, 29, 2016, ...","[-0.33317780570797084, 0.7364339818247828, -0...."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,"[videos, 15, civilians, killed, single, us, ai...","[-0.49488635880524096, 0.4494026989028818, -0...."
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,"[print, iranian, woman, sentenced, six, years,...","[-0.23221260471939204, 0.4406671484178874, -0...."


In [27]:
X['embedding'][0].shape

(100,)

#### prepare Data for trainning

In [28]:
import numpy as np
X_final=np.array(X['embedding'])
y_final=np.array(y)